# EDA – 이름 표기 불일치 (NER 기반)

이 노트북은 `code/name_ner_analysis.py`로 생성한 결과를 바탕으로,
- summary에서 추출된 사람 이름(`summary_persons`)
- dialogue에 등장하지 않는 이름(`unmatched_persons`)
- 불일치 비율(`has_unmatched`)

을 **표 + 그래프**로 확인하기 위한 EDA용 노트북입니다.

> 먼저 터미널에서 아래 스크립트를 실행해 두면 좋습니다.
> ```bash
> PYTHONPATH=src python code/name_ner_analysis.py --n_samples 0
> ```
> 그러면 `analysis/train_ner_name_analysis.csv`, `analysis/dev_ner_name_analysis.csv`가 생성됩니다.

## 1. 분석 결과 로딩

- `analysis/train_ner_name_analysis.csv`, `analysis/dev_ner_name_analysis.csv`를 불러옵니다.
- 파일이 없으면 친절한 에러 메시지를 출력합니다.

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

ANALYSIS_DIR = Path("analysis")
TRAIN_ANAL_PATH = ANALYSIS_DIR / "train_ner_name_analysis.csv"
DEV_ANAL_PATH = ANALYSIS_DIR / "dev_ner_name_analysis.csv"

missing = [p for p in [TRAIN_ANAL_PATH, DEV_ANAL_PATH] if not p.exists()]
if missing:
    raise FileNotFoundError(
        f"다음 분석 파일을 찾을 수 없습니다: {', '.join(str(p) for p in missing)}\n"
        "- 먼저 터미널에서 `PYTHONPATH=src python code/name_ner_analysis.py` 를 실행해 주세요."
    )

train_ner = pd.read_csv(TRAIN_ANAL_PATH)
dev_ner = pd.read_csv(DEV_ANAL_PATH)

train_ner.head(), dev_ner.head()

FileNotFoundError: 다음 분석 파일을 찾을 수 없습니다: analysis/train_ner_name_analysis.csv, analysis/dev_ner_name_analysis.csv
- 먼저 터미널에서 `PYTHONPATH=src python code/name_ner_analysis.py` 를 실행해 주세요.

## 2. 불일치 비율 (train vs dev)

- `has_unmatched` 비율을 train/dev 간에 비교합니다.
- 막대 그래프로 "요약에는 있지만 대화에는 없는 이름"이 얼마나 자주 나오는지 한눈에 봅니다.

In [ ]:
# train/dev 별 이름 불일치 비율을 계산하고 시각화합니다.
def mismatch_stats(df: pd.DataFrame) -> dict:
    total = len(df)
    num_unmatched = df["has_unmatched"].sum()
    ratio = (num_unmatched / total * 100) if total else 0.0
    return {"total": total, "num_unmatched": num_unmatched, "ratio": ratio}

train_stats = mismatch_stats(train_ner)
dev_stats = mismatch_stats(dev_ner)

summary_df = pd.DataFrame([
    {"split": "train", **train_stats},
    {"split": "dev", **dev_stats},
])
summary_df

In [ ]:
plt.figure(figsize=(5, 4))
sns.barplot(data=summary_df, x="split", y="ratio")
plt.title("이름 불일치 비율 (has_unmatched=True) [%]")
plt.ylabel("비율 (%)")
plt.xlabel("split")
plt.ylim(0, max(summary_df["ratio"]) * 1.2 if len(summary_df) else 1)
plt.show()

## 3. summary_persons / unmatched_persons 길이 분포

- 요약에서 몇 명의 사람 이름을 언급하는지(`summary_persons` 길이)
- 그 중 몇 명이 dialogue에 등장하지 않는지(`unmatched_persons` 길이)

를 길이 기준 히스토그램으로 확인합니다.

In [ ]:
# 리스트 컬럼에서 길이(len)를 계산하는 헬퍼
def list_len_safe(x):
    try:
        return len(eval(x)) if isinstance(x, str) else len(x)
    except Exception:
        return 0

for df in (train_ner, dev_ner):
    if not isinstance(df.get("summary_persons").iloc[0], list):
        df["summary_persons_len"] = df["summary_persons"].map(list_len_safe)
        df["unmatched_persons_len"] = df["unmatched_persons"].map(list_len_safe)
    else:
        df["summary_persons_len"] = df["summary_persons"].map(len)
        df["unmatched_persons_len"] = df["unmatched_persons"].map(len)

plt.figure(figsize=(8, 4))
sns.histplot(train_ner["summary_persons_len"], bins=10, kde=False, color="blue", label="train")
sns.histplot(dev_ner["summary_persons_len"], bins=10, kde=False, color="orange", label="dev", alpha=0.6)
plt.title("summary에서 추출된 사람 이름 수 분포")
plt.xlabel("summary_persons_len")
plt.ylabel("count")
plt.legend()
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(train_ner["unmatched_persons_len"], bins=10, kde=False, color="blue", label="train")
sns.histplot(dev_ner["unmatched_persons_len"], bins=10, kde=False, color="orange", label="dev", alpha=0.6)
plt.title("요약에는 있지만 대화에는 없는 이름 수 분포")
plt.xlabel("unmatched_persons_len")
plt.ylabel("count")
plt.legend()
plt.show()

## 4. 이름 불일치 예시 테이블

- 실제로 어떤 샘플에서 이름 불일치가 발생하는지, 표 형식으로 확인합니다.
- 상위 N개만 잘라서 보고, 필요하면 N을 늘려가며 확인할 수 있습니다.

In [ ]:
N = 10  # 보고 싶은 예시 개수

train_examples = train_ner[train_ner["has_unmatched"]].head(N).copy()
dev_examples = dev_ner[dev_ner["has_unmatched"]].head(N).copy()

cols = ["fname", "dialogue", "summary", "summary_persons", "unmatched_persons"]

print("### TRAIN 불일치 예시 ###")
train_examples[cols]

In [ ]:
print("### DEV 불일치 예시 ###")
dev_examples[cols]